In [43]:
from datasets import load_dataset
import numpy as np
import pandas as pd
import polars as pl
from sklearn.metrics import average_precision_score
from tqdm import tqdm

In [44]:
metadata = pd.read_csv("Enformer_metadata.csv")
metadata["name2"] = "feature_" + metadata.index.astype(str)
metadata

,name,file,clip,scale,sum_stat,description,assay,sample,name2
0,ENCFF833POA,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:cerebellum male adult (27 years) and mal...,DNASE,cerebellum male adult (27 years) and male adul...,feature_0
1,ENCFF110QGM,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:frontal cortex male adult (27 years) and...,DNASE,frontal cortex male adult (27 years) and male ...,feature_1
2,ENCFF880MKD,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:chorion,DNASE,chorion,feature_2
3,ENCFF463ZLQ,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:Ishikawa treated with 0.02% dimethyl sul...,DNASE,Ishikawa treated with 0.02% dimethyl sulfoxide...,feature_3
4,ENCFF890OGQ,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:GM03348,DNASE,GM03348,feature_4
...,...,...,...,...,...,...,...,...,...
5308,CNhs14239,/home/drk/tillage/datasets/human/cage/fantom/C...,384,1,sum,CAGE:epithelioid sarcoma cell line:HS-ES-2R,CAGE,epithelioid sarcoma cell line:HS-ES-2R,feature_5308
5309,CNhs14240,/home/drk/tillage/datasets/human/cage/fantom/C...,384,1,sum,CAGE:squamous cell lung carcinoma cell line:RE...,CAGE,squamous cell lung carcinoma cell line:RERF-LC-AI,feature_5309
5310,CNhs14241,/home/drk/tillage/datasets/human/cage/fantom/C...,384,1,sum,CAGE:gastric cancer cell line:GSS,CAGE,gastric cancer cell line:GSS,feature_5310
5311,CNhs14244,/home/drk/tillage/datasets/human/cage/fantom/C...,384,1,sum,CAGE:carcinoid cell line:NCI-H727,CAGE,carcinoid cell line:NCI-H727,feature_5311


In [45]:
dataset_path = "gonzalobenegas/siraj_eqtl_highpip"
V = load_dataset(dataset_path, split="test").to_pandas()
V

,chrom,pos,ref,alt,label
0,1,14677,G,A,True
1,1,63697,T,C,True
2,1,64764,C,T,True
3,1,108230,C,T,True
4,1,108826,G,C,True
...,...,...,...,...,...
27636,X,155150813,C,A,True
27637,X,155179766,A,G,False
27638,X,155189536,A,G,False
27639,X,155744353,G,T,True


In [46]:
features = "Enformer_absDelta"
df = pd.read_parquet(f"https://huggingface.co/datasets/{dataset_path}/resolve/main/features/{features}.parquet")
feature_cols = df.columns
V = pd.concat([V, df], axis=1)
V = V.dropna()
V

,chrom,pos,ref,alt,label,feature_0,feature_1,feature_2,feature_3,feature_4,...,feature_5303,feature_5304,feature_5305,feature_5306,feature_5307,feature_5308,feature_5309,feature_5310,feature_5311,feature_5312
3,1,108230,C,T,True,0.000024,0.000026,0.000022,3.206730e-05,0.000022,...,0.000033,1.049042e-05,5.602837e-06,4.053116e-06,1.788139e-06,1.013279e-06,8.940697e-07,0.000002,1.096725e-05,0.000003
5,1,135203,G,A,True,0.000022,0.000021,0.000034,2.431870e-05,0.000015,...,0.000031,1.138449e-05,4.470348e-06,2.622604e-06,9.536743e-07,1.788139e-07,1.430511e-06,0.000002,1.472235e-05,0.000006
8,1,281912,C,G,True,0.000018,0.000016,0.000011,2.509356e-05,0.000019,...,0.000024,2.443790e-06,1.847744e-06,1.728535e-06,3.218651e-06,4.112720e-06,2.384186e-06,0.000002,3.379583e-05,0.000025
9,1,286747,A,G,True,0.000004,0.000005,0.000001,7.748604e-06,0.000004,...,0.000002,7.152557e-07,3.576279e-07,8.344650e-07,3.933907e-06,5.960464e-08,4.172325e-07,0.000002,8.344650e-07,0.000005
10,1,586844,G,A,True,0.000147,0.000163,0.000143,2.260208e-04,0.000131,...,0.000097,9.149313e-05,2.640486e-05,4.595518e-05,4.053116e-06,5.245209e-06,4.649162e-06,0.000004,1.072884e-04,0.000014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26944,22,50733265,G,A,True,0.000037,0.000014,0.000145,2.861023e-06,0.000027,...,0.000179,1.680851e-05,9.357929e-06,4.822016e-05,2.324581e-06,1.490116e-06,7.748604e-07,0.000007,1.788139e-06,0.000018
26945,22,50759148,G,C,False,0.000035,0.000006,0.000021,5.960464e-07,0.000004,...,0.000230,1.357794e-04,3.737211e-05,2.712011e-05,1.847744e-05,9.238720e-06,1.233816e-05,0.000007,8.440018e-05,0.000045
26946,22,50779949,G,C,True,0.000060,0.000016,0.000006,2.264977e-06,0.000063,...,0.000074,4.708767e-05,3.814697e-06,9.119511e-06,3.170967e-05,1.668930e-05,2.861023e-05,0.000010,6.908178e-05,0.000087
26947,22,50781276,G,A,False,0.000440,0.000338,0.000233,2.532005e-04,0.000607,...,0.002291,2.424240e-03,6.079674e-04,2.011061e-04,3.337860e-04,1.199841e-04,3.535748e-04,0.000173,2.040863e-03,0.001203


In [47]:
res = []
for c in tqdm(feature_cols):
    res.append([c, average_precision_score(V["label"], V[c])])
res = pd.DataFrame(res, columns=["feature", "AUPRC"]).sort_values("AUPRC", ascending=False)
res

100%|███████████████████████████████████████████████████████| 5313/5313 [00:33<00:00, 156.62it/s]


,feature,AUPRC
3028,feature_3028,0.703540
2713,feature_2713,0.700314
2498,feature_2498,0.700034
3108,feature_3108,0.699989
606,feature_606,0.699295
...,...,...
2547,feature_2547,0.576232
4647,feature_4647,0.574504
2693,feature_2693,0.571892
4648,feature_4648,0.569655


In [48]:
res = res.merge(metadata, left_on="feature", right_on="name2")
res

,feature,AUPRC,name,file,clip,scale,sum_stat,description,assay,sample,name2
0,feature_3028,0.703540,ENCFF515MXQ,/home/drk/tillage/datasets/human/chip/encode/E...,32,2,mean,CHIP:H3K4me3:esophagus squamous epithelium fem...,CHIP,H3K4me3:esophagus squamous epithelium female a...,feature_3028
1,feature_2713,0.700314,ENCFF382HRD,/home/drk/tillage/datasets/human/chip/encode/E...,32,2,mean,CHIP:H3K4me3:breast epithelium female adult (5...,CHIP,H3K4me3:breast epithelium female adult (53 years),feature_2713
2,feature_2498,0.700034,ENCFF397IIG,/home/drk/tillage/datasets/human/chip/encode/E...,32,2,mean,CHIP:H3K4me3:coronary artery female adult (53 ...,CHIP,H3K4me3:coronary artery female adult (53 years),feature_2498
3,feature_3108,0.699989,ENCFF629RAA,/home/drk/tillage/datasets/human/chip/encode/E...,32,2,mean,CHIP:H3K27ac:lung female adult (30 years),CHIP,H3K27ac:lung female adult (30 years),feature_3108
4,feature_606,0.699295,ENCFF561DJM,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:lower leg skin female adult (53 years),DNASE,lower leg skin female adult (53 years),feature_606
...,...,...,...,...,...,...,...,...,...,...,...
5308,feature_2547,0.576232,ENCFF720MFS,/home/drk/tillage/datasets/human/chip/encode/E...,32,2,mean,CHIP:H3K9me3:germinal matrix male embryo (20 w...,CHIP,H3K9me3:germinal matrix male embryo (20 weeks),feature_2547
5309,feature_4647,0.574504,GSM1684641,/home/drk/tillage/datasets/human/chip/geo/GSM1...,64,1,sum,"CHIP:CEBPb:ChIP-seq, InVitroCistromics_CEBPb_0...",CHIP,"CEBPb:ChIP-seq, InVitroCistromics_CEBPb_0.25uL...",feature_4647
5310,feature_2693,0.571892,ENCFF472JYR,/home/drk/tillage/datasets/human/chip/encode/E...,32,2,mean,CHIP:H3K9me3:adrenal gland male adult (54 years),CHIP,H3K9me3:adrenal gland male adult (54 years),feature_2693
5311,feature_4648,0.569655,GSM1684642,/home/drk/tillage/datasets/human/chip/geo/GSM1...,64,1,sum,"CHIP:CEBPb:ChIP-seq, InVitroCistromics_CEBPb_1...",CHIP,"CEBPb:ChIP-seq, InVitroCistromics_CEBPb_1.0uL ...",feature_4648


In [49]:
res[["AUPRC", "description"]].head()

,AUPRC,description
0,0.703540,CHIP:H3K4me3:esophagus squamous epithelium fem...
1,0.700314,CHIP:H3K4me3:breast epithelium female adult (5...
2,0.700034,CHIP:H3K4me3:coronary artery female adult (53 ...
3,0.699989,CHIP:H3K27ac:lung female adult (30 years)
4,0.699295,DNASE:lower leg skin female adult (53 years)


In [50]:
res.groupby("assay").AUPRC.mean().sort_values(ascending=False)

assay
DNASE    0.673601
ATAC     0.661990
CHIP     0.651279
CAGE     0.644678
Name: AUPRC, dtype: float64

## Aggregate features

In [51]:
assays = metadata.assay.unique()
assays

array(['DNASE', 'ATAC', 'CHIP', 'CAGE'], dtype=object)

In [53]:
features2 = []
for assay in assays:
    assay_cols = feature_cols[(metadata.assay==assay).values]
    for norm_ord in [2]:#[1, 2, np.inf]:
        f = f"{assay}_L{norm_ord}"
        V[f] = np.linalg.norm(V[assay_cols], ord=norm_ord, axis=1)
        features2.append(f)
V

,chrom,pos,ref,alt,label,feature_0,feature_1,feature_2,feature_3,feature_4,...,feature_5307,feature_5308,feature_5309,feature_5310,feature_5311,feature_5312,DNASE_L2,ATAC_L2,CHIP_L2,CAGE_L2
3,1,108230,C,T,True,0.000024,0.000026,0.000022,3.206730e-05,0.000022,...,1.788139e-06,1.013279e-06,8.940697e-07,0.000002,1.096725e-05,0.000003,0.000611,0.001416,0.045621,0.000551
5,1,135203,G,A,True,0.000022,0.000021,0.000034,2.431870e-05,0.000015,...,9.536743e-07,1.788139e-07,1.430511e-06,0.000002,1.472235e-05,0.000006,0.000472,0.000962,0.028978,0.000546
8,1,281912,C,G,True,0.000018,0.000016,0.000011,2.509356e-05,0.000019,...,3.218651e-06,4.112720e-06,2.384186e-06,0.000002,3.379583e-05,0.000025,0.000356,0.001228,0.018882,0.000419
9,1,286747,A,G,True,0.000004,0.000005,0.000001,7.748604e-06,0.000004,...,3.933907e-06,5.960464e-08,4.172325e-07,0.000002,8.344650e-07,0.000005,0.000142,0.001066,0.016235,0.000550
10,1,586844,G,A,True,0.000147,0.000163,0.000143,2.260208e-04,0.000131,...,4.053116e-06,5.245209e-06,4.649162e-06,0.000004,1.072884e-04,0.000014,0.002436,0.002796,0.103831,0.001863
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26944,22,50733265,G,A,True,0.000037,0.000014,0.000145,2.861023e-06,0.000027,...,2.324581e-06,1.490116e-06,7.748604e-07,0.000007,1.788139e-06,0.000018,0.000800,0.001313,0.033894,0.001708
26945,22,50759148,G,C,False,0.000035,0.000006,0.000021,5.960464e-07,0.000004,...,1.847744e-05,9.238720e-06,1.233816e-05,0.000007,8.440018e-05,0.000045,0.000251,0.000521,0.014698,0.002718
26946,22,50779949,G,C,True,0.000060,0.000016,0.000006,2.264977e-06,0.000063,...,3.170967e-05,1.668930e-05,2.861023e-05,0.000010,6.908178e-05,0.000087,0.001018,0.003766,0.033113,0.002983
26947,22,50781276,G,A,False,0.000440,0.000338,0.000233,2.532005e-04,0.000607,...,3.337860e-04,1.199841e-04,3.535748e-04,0.000173,2.040863e-03,0.001203,0.010976,0.008085,0.219951,0.050568


In [54]:
res2 = []
for c in features2:
    res2.append([c, average_precision_score(V["label"], V[c])])
res2 = pd.DataFrame(res2, columns=["feature", "AUPRC"]).sort_values("AUPRC", ascending=False)
res2

,feature,AUPRC
2,CHIP_L2,0.726559
0,DNASE_L2,0.722671
3,CAGE_L2,0.705801
1,ATAC_L2,0.686291


In [55]:
from sklearn.linear_model import LogisticRegression

X = V[features2]
y = V["label"]

clf = LogisticRegression(max_iter=1000).fit(X, y)
clf

LogisticRegression(max_iter=1000)

In [56]:
for i in range(len(features2)):
    print(features2[i], clf.coef_[0][i])

DNASE_L2 1.170910959772034
ATAC_L2 -0.7648403570840655
CHIP_L2 4.347632823905649
CAGE_L2 5.5072351654386456


In [57]:
pred = clf.predict_proba(X)[:, 1]
average_precision_score(y, pred)

0.7308174609312403